In [1]:
import dask.dataframe as dd
import pandas as pd 
import dask as d
from tqdm import tqdm, trange

In [35]:
import os 
# Turn all scenario .csv files into an array of pandas data frames
dfs = {}

#data folder should include csv's with all scenarios initial data from processDeploymentsToScenarios    
for file in tqdm(os.listdir("data")):
    if file.endswith(".csv"):
        filename = os.path.join("data", file)
        df = dd.read_csv(filename, dtype='str')
        newdf = df.compute()
        dfs[file[:-4]] = newdf

    
#Make a copy of the dfs array to drop columns including latitude and longitude
dfs_no_lat_lon = dfs.copy()


100%|██████████| 18/18 [00:25<00:00,  1.41s/it]


In [36]:
outfile_name = 'outfile_extended3.csv'

In [37]:
#Preprocessing work, not important

newCols = ['latitude', 'longitude']

scenarios = ['Open Access - Mid', 'Open Access - Low', 'Open Access - Current', 'Baseline - Mid',
            'Baseline - Current', 'Baseline - Low', 'Legacy - Current', 'Existing Social Acceptance - Mid',
            'Mid Social Acceptance - Mid', 'Mid Social Acceptance - Low', 'Low Social Acceptance - Mid',
            'Low Social Acceptance - Low', 'Radar Limits - Mid',
            'Smart Bat Curtailment - Mid', 'Blanket Bat Curtailment - Mid', 
            'Fed Land Exclusion - Mid', 'Limited Access - Mid']

excludedCols = ['','state', 'county', 
                'scenario', 'Unnamed: 0', 'Unnamed:0', 'scenario_abbrv',
                 'timezone','2016_30m_cdls_rev90m', 
               'blm_moderate_siting_considerations', 'reeds_region',
                'setting', 'reeds_ba', 'reeds_raz', 'cf_bin',
                'trans_type', 'fips','lcoe_bin',
               'blm_high_siting_considerations', 'non_listed_sensitive_bats',
               'te_esa_listed_bats', 'other_key_te_species',
                'latitude', 'longitude']

excludedCols_noLatLon = ['','state', 'county', 
                'scenario', 'Unnamed: 0', 'Unnamed:0', 'scenario_abbrv',
                'timezone','2016_30m_cdls_rev90m', 
               'blm_moderate_siting_considerations', 'reeds_region',
                'setting', 'reeds_ba', 'reeds_raz', 'cf_bin',
                'trans_type', 'fips','lcoe_bin',
               'blm_high_siting_considerations', 'non_listed_sensitive_bats',
               'te_esa_listed_bats', 'other_key_te_species']


df = dfs[scenarios[0]]
cols = list(df.columns)
#print(dfs)
for colName in excludedCols:
    try:
        cols.remove(colName)
    except:
        print(colName)

for df in dfs:
    for colName in excludedCols_noLatLon:
        try:
            dfs[df] = dfs[df].drop(colName, axis=1)
        except:
            print(colName + " not found.")
            
dfs_no_lat_lon = dfs.copy()
for scenario in dfs_no_lat_lon:
    dfs_no_lat_lon[scenario].drop(['latitude', 'longitude'], axis=1)
        



 not found.
 not found.
 not found.
 not found.
 not found.
 not found.
 not found.
 not found.
 not found.
 not found.
 not found.
 not found.
 not found.
 not found.
 not found.
 not found.
 not found.


In [38]:
#Create list of latitudes and longitudes from all points in all files
locations = set()
for scenarioName in tqdm(dfs):
    df = dfs[scenarioName]
    lats = df['latitude'].to_numpy()
    lons = df['longitude'].to_numpy()
    for i in range(len(lats)):
        loc = tuple([lats[i], lons[i]])
        if(loc not in locations):
            locations.add(loc)
        else:
            pass
 

100%|██████████| 17/17 [00:00<00:00, 17.25it/s]


In [39]:
print(list(dfs_no_lat_lon['Baseline - Mid'].columns))

['sc_gid', 'n_gids', 'mean_cf', 'mean_lcoe', 'mean_res', 'capacity', 'area_sq_km', 'latitude', 'longitude', 'elevation', 'usa_mrlc_nlcd2011', 'conus_elev_resampled', 'fed_land_owner', 'lbnl_convex_hull_existing_farms_2018', 'conus_roads_proximity_int32', 'nlcd_2016_treecanopy_2019_08_31_rev90m', 'nsrdb_mym_ghi', 'nsrdb_mym_cf1axis', 'conus_rail_90_proximity_int32', 'wtk_mym_ws80', 'transmission_multiplier', 'trans_cap_cost', 'dist_mi', 'lcot', 'total_lcoe', 'gw', 'capacity_bin_mw', 'state_fips', 'lbnl_region', 'population', 'wind_class', 'cum_cap_gw', 'mwh_mw', 'mwh', 'road_bin_km', 'rail_bin_km', 'count', 'technology']


In [40]:
#Drop unecessary rows
#Getting key errors is not a big deal here - just means it's already dropped
for scenarioName in tqdm(dfs_no_lat_lon):
    df = dfs_no_lat_lon[scenarioName]
    #print(list(df.columns))
    for colName in excludedCols_noLatLon:

        try:
            dfs_no_lat_lon[scenarioName] = df.drop(colName, axis=1)
        except KeyError:
            print("Key Error For " + colName)



100%|██████████| 17/17 [00:00<00:00, 193.47it/s]

Key Error For 
Key Error For state
Key Error For county
Key Error For scenario
Key Error For Unnamed: 0
Key Error For Unnamed:0
Key Error For scenario_abbrv
Key Error For timezone
Key Error For 2016_30m_cdls_rev90m
Key Error For blm_moderate_siting_considerations
Key Error For reeds_region
Key Error For setting
Key Error For reeds_ba
Key Error For reeds_raz
Key Error For cf_bin
Key Error For trans_type
Key Error For fips
Key Error For lcoe_bin
Key Error For blm_high_siting_considerations
Key Error For non_listed_sensitive_bats
Key Error For te_esa_listed_bats
Key Error For other_key_te_species
Key Error For 
Key Error For state
Key Error For county
Key Error For scenario
Key Error For Unnamed: 0
Key Error For Unnamed:0
Key Error For scenario_abbrv
Key Error For timezone
Key Error For 2016_30m_cdls_rev90m
Key Error For blm_moderate_siting_considerations
Key Error For reeds_region
Key Error For setting
Key Error For reeds_ba
Key Error For reeds_raz
Key Error For cf_bin
Key Error For tran

In [41]:

#Create a 'data' dictionary that will be able to access specific lat/lon locations for 
# specific scenarios for fast access

#Set variable region = 0 if you want all regions, or set to a region you want to limit against


# data = {  scenarioName :  { str(lat)+str(lon) : row, str(lat)+str(lon) : row, ...  }, scenarioName : ...  }
data = {}

for scenarioName in tqdm(dfs):
    data[scenarioName] = {}
    df = dfs[scenarioName]
    for index, row in df.iterrows():
        
        
        name = str(row['latitude']) + str(row['longitude'])
        data[scenarioName][name] = row



100%|██████████| 17/17 [01:45<00:00,  6.23s/it]


In [43]:



newDataColNames = ['latitude', 'longitude']

#Create array of column names for final csv file
for scenarioName in dfs:
    for col in cols:
        newDataColNames.append(scenarioName + ':' + col)

#Create empty dataframe and make a list out of the 'locations' set to iterate through
newDataFrame = pd.DataFrame(columns=newDataColNames, dtype=object)
columnsWritten=False
locations = list(locations)

#Loop through all locations and scenario names. 
# Access the data[scenarioName][location] row from the data dictionary
# Append that information the the existing dataframe.  Write out every 200 iterations for performance reasons
for i in trange(len(locations)):
    lat = float(locations[i][0])
    lon = float(locations[i][1])
    loc = str(lat) + str(lon)
    
    #Generate a new dictionary that will become the new dataframe
    newData = {'latitude' : lat, 'longitude' : lon}
    
    #Go through each of the 17 scenario dataframes, pulling info from each
    for scenarioName in dfs_no_lat_lon:
        
        #Get row
        try:
            row = data[scenarioName][loc]
 
        except KeyError:
            #Set row to 0 if that scenario does not have data for location: loc
            row = 0
            
        for col in cols:
            newColName = scenarioName + ':' + col
            try:
                #Add data from the row to the newData dictoinary
                x = row[col]
                newData[newColName] = x
                
            except TypeError:
                #Add empty data if row = 0
                newData[newColName] = ''
            
    #Create a new data frame from the new data and concatenate it with old data frame
    n = pd.DataFrame(newData, index=[0])
    newDataFrame = pd.concat([newDataFrame, n], ignore_index=True)

    #Write out every 200 iterations
    if(i%200==0 and i != 0):
        if(not columnsWritten):
            
            newDataFrame.to_csv(outfile_name,mode='w', index=False)
            newDataFrame = pd.DataFrame(columns=newDataColNames)
            columnsWritten=True
        else:
            newDataFrame.to_csv(outfile_name,mode='a', header=False, index=False)
            newDataFrame = pd.DataFrame(columns=newDataColNames)


#Final write out
newDataFrame.to_csv(outfile_name,mode='a', header=False, index=False)  

100%|██████████| 66000/66000 [1:12:36<00:00, 15.15it/s]


In [61]:
#Creates regional datasets
#newDataFrame.to_csv(outfile_name,mode='a', header=False, index=False)  
regions = ['Interior', 'Northeast', 'Southeast', 'West', 'Great Lakes']
region = regions[0]


    
df = dd.read_csv(outfile_name, dtype='str')
newdf = df.compute()
for region in tqdm(regions):
    outfile_name = 'data_extended_' + region + "2.csv"
    df_region = newdf[newdf['Open Access - Mid:lbnl_region'] == region]
    df_region.to_csv(outfile_name, mode='w', index=False)

100%|██████████| 5/5 [00:22<00:00,  4.48s/it]


In [12]:
d = pd.read_csv('outfile_extended2.csv')

In [25]:
#Add s_gids
df_s_gids = dfs['Existing Social Acceptance - Mid'].copy()
df_s_gids.drop(df.columns.difference(['latitude', 'longitude', 'sc_gid']), 1, inplace=True)
new = pd.concat([df_s_gids, d], sort=False, join="inner")